# Peavey VIP-2 USB Scanner

![Peavey VIP-2](vip2-small.png)

Other relevant documentation includes:

* Peavey Vypyr Pro MIDI Specification (not entirely the same, but ideas are shared) https://assets.peavey.com/literature/additional/118745_31036.pdf
* My own footswitch controller for Arduino https://github.com/aughey/peavey_footswitch/blob/master/peavey_footswitch.ino
* Another DIY footswitch http://www.claytonfelt.com/peavey-vypyr-vip-amp-diy-midi-footswitch/
* Link to a reverse engineered MIDI spec for the commercial footswitch https://www.vguitarforums.com/smf/index.php?topic=3119.0

The trick here is to reverse engineer data captured from wireshark of the Peavey supplied Windows application.  The relevant
capture is below.

![Wireshark](wireshark.PNG)

This is the message sent by the application to retrieve the current amp state.  In response to a preset change, the
app sends this syex message and the amp response with a long sysex string.

Unfortunately, Wireshark is displaying this USB capture strangely.  It's interjecting 0x04 values every 4 values, so
the capture data 0x04 0xf0 0x00 0x00 0x04 0x1b 0x12 0x00 0x04, with the 0x04's stripped out is really, 0xf0, 0x00, 0x00, 0x1b, 0x12, 0x00.  I can't explain why wireshark is showing it this way, but it is.  Strangely, at the end there is 0x05 instead of 0x04 insertted.

In [1]:
import mido
import sys
import time
import json

Get Find the midi ports with VYPYR in in

In [2]:
input = None
output = None

for name in mido.get_output_names():
  if name.startswith("VYPYR"):
    output = mido.open_output(name)
    print("Output Opened: ",name)
    
for name in mido.get_input_names():
  if name.startswith("VYPYR"):
    input = mido.open_input(name)
    print("Input Opened:  ",name)


Output Opened:  VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0
Input Opened:   VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0


## Methods for manipulating the device over MIDI

In [3]:
prefix = [ 0x00, 0x00, 0x1B, 0x12, 0x00]
def send_control(id,value):
    output.send(msg = mido.Message("control_change", control=id, value=value))

def send_query():
    flush()
    msg = mido.Message('sysex', data= prefix + [0x63, 0x7f, 0x7f])
    output.send(msg)
    
def flush():
    for msg in input.iter_pending():
        None
        
def read_data():
    for msg in input.iter_pending():
       print(msg)
       print(msg.hex())
        
def read_block(type = None):
    for msg in input:
       if type == None or type == msg.type:
           return msg
        
def read_conf():
    send_query();
    return read_block(type='sysex')

def wait_for_data(t):
    start = time.time()
    while time.time() - start < t:
        msg = input.poll()
        print("a")
        if msg:
            return msg
    return None

read_data()
flush()

Mostly reference here for how to 

In [4]:
msg = mido.Message('note_on',note=0x10, channel=1)
output.send(msg);
msg = mido.Message('note_off',note=0x10, channel=1)
output.send(msg);


If we really need to write raw data, the _rt method is available, but thankfully this sysex approach is going to work.

In [5]:
output._rt

## Inquiry command - Get version

This sysex message is documented in the Vypyr Pro manual as an Inquiry command.  

The response to a standard inquiry is: F0 7E 7F 06 02 00 00 1B 30 00 00 00 ww xx yy zz F7

where:

'ww' is 41 ('A' amp mode), 44 ('D' demo mode), 54 ('T' tuner mode) or 42 ('B' bootcode -
software update mode)

'xx' is the major version number (30 or 31 for 0.0 or 1.0)

'yy' and 'zz' are the minor version digits (30..39 for a range of .00 to .99)

Perhaps this can be used to query the type of amp before sending other commands to check for compatibility.

In [6]:
#msg = mido.Message('sysex', data=[0x00,0x00,0x04,0x1b,0x12,0x00,0x04,0x63,0x7f,0x7f,0x05])
flush()
msg = mido.Message('sysex', data=[ 0x7E, 0x7F, 0x06, 0x01])
print(msg.hex())

F0 7E 7F 06 01 F7


In [7]:
output.send(msg)
inquiry_reply = read_block()
print(inquiry_reply.hex());

F0 7E 00 06 02 00 00 1B 12 00 09 00 30 31 36 31 F7


In [8]:
major = 0
wwxxyyzz = inquiry_reply.data[11:]
if wwxxyyzz[1] == 0x31:
    major = 1
minor = (wwxxyyzz[2] - 0x30) + (wwxxyyzz[3] - 0x30) * 10
print("Version: " + str(major) + "." + str(minor))

Version: 1.16


# Read the configuration block

In [9]:
send_query()
print(read_block().hex())

F0 00 00 1B 12 09 63 7F 7F 00 02 07 0F 07 0F 07 0F 03 03 04 04 00 02 00 06 00 0C 00 02 00 02 03 07 03 07 00 02 02 08 00 02 00 00 00 0A 00 02 00 02 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 00 00 00 00 00 00 00 00 04 0C 04 05 04 01 04 04 02 00 04 07 04 01 04 09 04 0E 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 0A 05 F7


## The result of this should be true.  Just reading the block twice

In [10]:
send_query()
a = read_block()
time.sleep(1)
send_query()
b = read_block()
a == b

True

We find a control by setting the value of the control to 1, reading it, changing the control value to 2, and reading
the configuration a second time.  One or more fields should change.  The changed field(s) are the values affected by this
control.

In [11]:
def find_control(id):
    send_control(id,2);
    a = read_conf()
    send_control(id,126);
    b = read_conf()
    i=0
    count = 0
    changed = []
    myindex = None
    for v in zip(a.data,b.data):
        if v[0] != v[1]:
            print("  Index: ",i," ",hex(v[0])," != ",hex(v[1]))
            if v[0] == 2 and v[1] == 126:
                myindex = i
            changed.append(i)
            count += 1
        i=i+1
    return { 'myindex': myindex, 'changed': changed, 'count': count }

In [12]:
flush()
find_control(20)

  Index:  18   0x0  !=  0x7
  Index:  19   0x2  !=  0xe


{'changed': [18, 19], 'count': 2, 'myindex': None}

In [13]:
find_control(21)

  Index:  34   0x0  !=  0x7
  Index:  35   0x2  !=  0xe


{'changed': [34, 35], 'count': 2, 'myindex': None}

In [14]:
find_control(1)

{'changed': [], 'count': 0, 'myindex': None}

## Documentation:

Below are the control values for the knobs on the VIP-2.  These were determined by rotating knobs and watching the MIDI stream.

### Control Change

* pregain = 16
* low = 17
* mid = 18
* high = 19
* postgain = 20

* P1 = 27
* P2 = 26
* Delay Feedback = 21
* Delay Level = 23
* Reverb = 31

* Effect = 10
* Amp = 8
* Inst/Stomp = 11

![Front Panel](frontpanel.png)


In [15]:
known_controls = {
    "amptype" : 12,
    "Pre-Gain" : 16,
    "Low": 17,
    "Mid" : 18,
    "High" : 19,
    "Post-Gain" : 20,
    "P1" : 27,
    "P2" : 26,
    "Delay Feedback" : 21,
    "Delay Level" : 23,
    "Reverb" : 31,
    "Effect" : 10,
    "Amp" : 8,
    "Inst/Stomp" : 11
}

# Loop through all controls from 1 to 32 inclusive

Print out the configuration indices that change.  The controls that change should correspond to the known controls above.

More importantly, this searches for controls that are represented in the configuration that we do not have in our known
controls list.  Any control that hits this warning should be checked to see what it affects.

In [16]:
for control in range(1,32):
    print("Control: ",control)
    if find_control(control)['count'] > 0:
        # Make sure this is one of our known controls
        if not control in known_controls.values(): # Isn't that beatifully written?
            print("  Warning: this control changed something in the config, but isn't a known control.")

Control:  1
Control:  2
Control:  3
Control:  4
Control:  5
Control:  6
Control:  7
Control:  8
  Index:  9   0x2  !=  0x5
  Index:  10   0x7  !=  0x3
  Index:  11   0xf  !=  0xe
  Index:  12   0x7  !=  0x6
  Index:  13   0xf  !=  0xa
  Index:  14   0x7  !=  0x4
  Index:  15   0xf  !=  0x6
  Index:  16   0x3  !=  0x7
  Index:  17   0x3  !=  0xf
  Index:  18   0x7  !=  0x5
  Index:  19   0xe  !=  0x4
Control:  9
Control:  10
  Index:  25   0xc  !=  0x5
  Index:  26   0x0  !=  0x5
  Index:  28   0x0  !=  0x5
Control:  11
  Index:  23   0x6  !=  0x5
  Index:  31   0x7  !=  0x4
  Index:  33   0x7  !=  0x4
  Index:  43   0xa  !=  0x0
Control:  12
  Index:  8   0x1  !=  0x0
  Index:  9   0xd  !=  0x5
  Index:  10   0x4  !=  0x3
  Index:  11   0x4  !=  0xe
  Index:  12   0x2  !=  0x6
  Index:  13   0x5  !=  0xa
  Index:  14   0x5  !=  0x4
  Index:  15   0x4  !=  0x6
  Index:  16   0x6  !=  0x7
  Index:  17   0xa  !=  0xf
  Index:  19   0xa  !=  0x4
Control:  13
Control:  14
Control:  15
Contr

In [17]:
for control,control_id in known_controls.items():
    print("Control: ",control,"(",control_id,")")
    info = find_control(control_id)
    if 0 == info['count']:
        print("  Warning: this is a known control, but we didn't see a change in the configuration")
    known_controls[control] = { 'control': control_id, 'config_index': info['myindex']}

Control:  Mid ( 18 )
  Index:  12   0x0  !=  0x7
  Index:  13   0x2  !=  0xe
Control:  amptype ( 12 )
  Index:  8   0x1  !=  0x0
  Index:  9   0xd  !=  0x5
  Index:  10   0x4  !=  0x7
  Index:  11   0x4  !=  0xe
  Index:  12   0x2  !=  0x7
  Index:  13   0x5  !=  0xe
  Index:  14   0x5  !=  0x7
  Index:  15   0x4  !=  0xe
  Index:  16   0x6  !=  0x7
  Index:  17   0xa  !=  0xe
  Index:  18   0x5  !=  0x7
  Index:  19   0xa  !=  0xe
Control:  P2 ( 26 )
  Index:  28   0x0  !=  0x7
  Index:  29   0x2  !=  0xe
Control:  Reverb ( 31 )
  Index:  20   0x0  !=  0x7
  Index:  21   0x2  !=  0xe
Control:  Pre-Gain ( 16 )
  Index:  16   0x0  !=  0x7
  Index:  17   0x2  !=  0xe
Control:  Delay Level ( 23 )
  Index:  38   0x0  !=  0x7
  Index:  39   0x2  !=  0xe
Control:  Amp ( 8 )
  Index:  9   0x2  !=  0x5
  Index:  11   0xf  !=  0xe
  Index:  13   0xf  !=  0xe
  Index:  15   0xf  !=  0xe
  Index:  16   0x3  !=  0x7
  Index:  17   0x3  !=  0xe
Control:  Post-Gain ( 20 )
  Index:  18   0x0  !=  0x7

In [18]:
print(json.dumps(known_controls))

{"Mid": {"control": 18, "config_index": null}, "amptype": {"control": 12, "config_index": null}, "P2": {"control": 26, "config_index": null}, "Reverb": {"control": 31, "config_index": null}, "Pre-Gain": {"control": 16, "config_index": null}, "Delay Level": {"control": 23, "config_index": null}, "Amp": {"control": 8, "config_index": null}, "Post-Gain": {"control": 20, "config_index": null}, "Delay Feedback": {"control": 21, "config_index": null}, "Effect": {"control": 10, "config_index": null}, "Low": {"control": 17, "config_index": null}, "High": {"control": 19, "config_index": null}, "P1": {"control": 27, "config_index": null}, "Inst/Stomp": {"control": 11, "config_index": null}}
